## Create a sample database

The schema was borrowed from this medium article https://medium.com/snowflake/a-sales-assistant-powered-by-a-react-agent-and-llama-2-on-snowflake-95f6331d43db.

In [9]:
database_schema = """

DROP TABLE IF EXISTS PRODUCTS;
CREATE TABLE PRODUCTS (
    SKU INT,
    DESCRIPTION STRING
);
DROP TABLE IF EXISTS INVENTORY;
CREATE TABLE INVENTORY (
    SKU INT,
    QUANTITY INT,
    WAREHOUSE STRING
);

INSERT INTO PRODUCTS(SKU,DESCRIPTION) VALUES (100,'short sleeve cotton shirt blue');
INSERT INTO PRODUCTS(SKU,DESCRIPTION) VALUES (200,'long sleeve flanelle shirt red');
INSERT INTO PRODUCTS(SKU,DESCRIPTION) VALUES (300, 'Black Running Shoes size 11');
INSERT INTO PRODUCTS(SKU,DESCRIPTION) VALUES (400, 'Sweat Pants Grey Men');
INSERT INTO PRODUCTS(SKU,DESCRIPTION) VALUES (500, 'Crop top green Women');
INSERT INTO PRODUCTS(SKU,DESCRIPTION) VALUES (600, 'Long Skirt Blue');
INSERT INTO PRODUCTS(SKU,DESCRIPTION) VALUES (700, 'Summer hat kids Orange');
INSERT INTO PRODUCTS(SKU,DESCRIPTION) VALUES (800, 'Sunglasses Unisex black');
INSERT INTO PRODUCTS(SKU,DESCRIPTION) VALUES (900, 'Jeans Black');
INSERT INTO PRODUCTS(SKU,DESCRIPTION) VALUES (1000, 'Kaki cotton pants summer women');

INSERT INTO INVENTORY VALUES (100, 154,'New York');
INSERT INTO INVENTORY VALUES (200, 245, 'Seattle');
INSERT INTO INVENTORY VALUES (300, 364, 'San Francisco');
INSERT INTO INVENTORY VALUES (400, 421, 'New York');
INSERT INTO INVENTORY VALUES (500, 532, 'Seattle');
INSERT INTO INVENTORY VALUES (600, 678, 'New York');
INSERT INTO INVENTORY VALUES (700, 741, 'San Francisco');
INSERT INTO INVENTORY VALUES (800, 845, 'New York');
INSERT INTO INVENTORY VALUES (900, 987, 'Seattle');
INSERT INTO INVENTORY VALUES (1000, 149, 'San Francisco');
"""

In [10]:
import sqlite3

connection_obj = sqlite3.connect('inventory.db')
# cursor object
cursor_obj = connection_obj.cursor()
cursor_obj.executescript(database_schema)
# Test inserted records
cursor_obj.execute("SELECT * from INVENTORY") 
# fetch all the data 
print(cursor_obj.fetchall())
# Another query
cursor_obj.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor_obj.fetchall())
connection_obj.close()

[(100, 154, 'New York'), (200, 245, 'Seattle'), (300, 364, 'San Francisco'), (400, 421, 'New York'), (500, 532, 'Seattle'), (600, 678, 'New York'), (700, 741, 'San Francisco'), (800, 845, 'New York'), (900, 987, 'Seattle'), (1000, 149, 'San Francisco')]
[('PRODUCTS',), ('INVENTORY',)]


In [12]:
import os

class DBTool():
    
    def __init__(self, database):
        
        if not os.path.isfile(database):
            print("Database does not exist")
            raise ValueError("Database does not exist")
        try:
            self.connection = sqlite3.connect(database)
            self.cursor = self.connection.cursor()
        except:
           print(f"Unable to open{database}") 
           raise ValueError(f"unable to open {database}")

        
        self.table_cols   = {}
        self.__get_tbls()
        self.__get_cols()
        self.connection.close()
    
    def get_string_signature(self):
        return_string = """
        Table names and their columns are listed below \n
        """
        
        table_count = 1
        for table, columns in self.table_cols.items():
            return_string+= str(table_count) + "\n"
            return_string+= f"Table name:{table} \n"
            for idx, col_name in enumerate(columns):
                return_string+="\t" + str(idx+1) + f" {col_name} \n"
            table_count+=1
        
        return return_string
            
    
    def __get_tbls(self):
        self.cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = self.cursor.fetchall()
        for (table,) in tables:
            self.table_cols[table] = []
    
    def __get_cols(self):
        for table in self.table_cols.keys():
            columns = self.cursor.execute(f"PRAGMA table_info({table})")
            for column in columns:
                self.table_cols[table].append(column[1])


db_tool = DBTool("inventory.db")
table_schema = db_tool.get_string_signature()
print(table_schema)


        Table names and their columns are listed below 

        1
Table name:PRODUCTS 
	1 SKU 
	2 DESCRIPTION 
2
Table name:INVENTORY 
	1 SKU 
	2 QUANTITY 
	3 WAREHOUSE 

